etude 1

(a)

$\Delta\sim\mathcal{N}(0, \sigma^2((X_0^TX_0)^{-1}+(X_1^TX_1)^{-1}))$

$\delta\sim\mathcal{N}(0, \sigma^2(2I-H_0-H_1))$

(b)

(c)

$A = ((X_0^TX_0)^{-1}+(X_1^TX_1)^{-1})^{-\frac{1}{2}}$


Eigendecomposition: $2I-H_0-H_1=Q\Lambda Q^T$

$M = {\Lambda^\dagger}^{\frac{1}{2}}Q^T$

(d)

$T=\frac{\frac{1}{d}\|A\Delta\|_2^2}{\frac{1}{n-r}\|M\delta\|_2^2}\sim F_{n-r}^d $

(e)

In [34]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from scipy.stats import f

In [6]:
df = pd.read_csv('maybe-its-nonlinear.csv', header=None, sep=' ')
X = df[[0, 1]].to_numpy()
# X = np.insert(X, 0, 1, axis=1)  # insert all ones column to the data matrix
y = df[2].to_numpy()

In [52]:
def sample_from_sphere(d):  # sample a random point on the unit d-sphere
    v = np.random.randn(d)
    return v / np.linalg.norm(v)

def random_projection_split(X):
    n, d  = X.shape[0] // 2, X.shape[1]
    v = sample_from_sphere(d)
    S0 = np.argsort(X @ v)[:n]
    S1 = np.argsort(X @ v)[n:]
    return S0, S1

def random_split(X):
    n = len(X) // 2
    permutation = np.random.permutation(len(X))
    return permutation[:n], permutation[n:]

In [36]:
def beta_hat(X, y):
    return np.linalg.solve(X.T @ X, X.T @ y)
    
def hat_matrix(X):
    return X @ np.linalg.solve(X.T @ X, X.T)

In [50]:
def test(X, y, split, alpha=0.05):
    # return False if reject, i.e. p value < alpha

    n, d = X.shape[0] // 2, X.shape[1]
    S0, S1 = split(X)
    X0, X1 = X[S0], X[S1]
    y0, y1 = y[S0], y[S1]

    beta_hat0, beta_hat1 = beta_hat(X0, y0), beta_hat(X1, y1)
    Delta =  beta_hat0 - beta_hat1

    H0, H1 = hat_matrix(X0), hat_matrix(X1)
    delta = (np.eye(n) - H0) @ y0 - (np.eye(n) - H1) @ y1

    eigenvalues, Q = np.linalg.eigh(np.linalg.inv(X0.T @ X0) + np.linalg.inv(X1.T @ X1))
    A = Q @ np.diag(1 / np.sqrt(eigenvalues)) @ Q.T

    eigenvalues, Q = np.linalg.eigh(2 * np.eye(n) - H0 - H1)
    M = np.diag(1 / np.sqrt(eigenvalues)) @ Q.T

    T = (
        (np.sum((A @ Delta) ** 2) / d) /
        (np.sum((M @ delta) ** 2) / n)
    )
    
    p_val = 1 - f.cdf(T, d, n)
    
    return p_val >= alpha

In [58]:
reject0, reject1 = 0, 0
for _ in range(1000):
    reject0 += 1 - test(X, y, random_projection_split)
    reject1 += 1 - test(X, y, random_split)

In [60]:
print(reject0 / 1000, reject1 / 1000)

1.0 0.067


The data exhibits nonlinearity by random hyperplane separation.

If we split randomly, with  0.067 $\approx\alpha$ fraction of the time we reject.